# Исследование данных

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import re
import ast
from Levenshtein import distance


In [2]:
# чтение файла
data = pd.read_csv('data/data.csv', sep=',')
# вывод первых двух строк
data.head(2)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"


In [3]:
#размерность таблицы
data.shape

(377185, 18)

In [4]:
#получение информации о столбцах
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

В большинстве признаков имеются пропуски в данных. Для наглядности выразим пропущенные значения в процентах.

In [5]:
percent_null = (data.isnull().sum() / len(data)) * 100
display(percent_null.sort_values(ascending=False))

private pool    98.891525
mls-id          93.387330
PrivatePool     89.312672
fireplace       72.662221
stories         39.958111
baths           28.192532
beds            24.200856
MlsId           17.731352
sqft            10.757851
status          10.583136
propertyType     9.208479
target           0.657767
city             0.009014
street           0.000530
zipcode          0.000000
schools          0.000000
state            0.000000
homeFacts        0.000000
dtype: float64

In [6]:
data.describe(include=['object'])

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
count,337267,4181,342452,377183,270847,377185,103114,377151,377185,336608,377185,285903,377185,226469,24942,40311,310305,374704
unique,159,1,1280,337076,229,321009,1652,2026,297365,25405,4549,1184,39,347,24907,2,232944,43939
top,for sale,Yes,single-family home,Address Not Disclosed,2 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",yes,Houston,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,32137,3 Beds,FL,1.0,No MLS#,yes,NO MLS,"$225,000"
freq,156104,4181,92206,672,52466,7174,50356,24442,4204,11854,2141,53459,115449,67454,3,28793,24,1462


# Исследование отдельных признаков

# 1. Status

Признак содержит 159 уникальных категорий

In [7]:
print('Доля (%) 50-ти наиболее весомых категорий: ' , round((data['status'].value_counts().nlargest(50)/len(data)*100).sum(),1))

Доля (%) 50-ти наиболее весомых категорий:  89.3


In [8]:
data['status'].value_counts().nlargest(50)

status
for sale                             156104
Active                               105207
For sale                              43465
foreclosure                            6426
New construction                       5475
Pending                                4702
Pre-foreclosure                        2119
Pre-foreclosure / auction              1560
P                                      1488
Under Contract Show                    1183
 / auction                              936
Under Contract   Showing                793
Active Under Contract                   718
New                                     690
Under Contract                          690
Contingent                              581
Price Change                            563
Auction                                 536
Foreclosed                              459
A Active                                443
for rent                                398
Foreclosure                             343
recently sold            

In [9]:
# Создаем словарь с заменами
replacements = {
    'foreclosed': 'foreclosure',
    'activated': 'active',
    'a active': 'active',
    'pre-foreclosure / auction': 'pre-foreclosure',
    'pre foreclosure auction': 'pre-foreclosure',
    'under contract show': 'under Contract',
    'under contract   showing': 'under Contract',
    'active under contract': 'under Contract',
    '/ auction': 'auction',
    'auction': 'auction',
    'auction - active': 'auction',
    'active auction': 'auction',
    'auction active': 'auction',
    'under contract backups':'backup contract',
    'active backup': 'backup contract',
    'pending continue to show': 'active with offer',
    'pending backup wanted': 'active with offer',
    'pending - taking backups': 'pending take backups'
}

# Приводим значения столбца "status" к нижнему регистру
data['status'] = data['status'].str.lower()

# Заменяем значения в столбце "status" с помощью словаря замен
data['status'].replace(replacements, inplace=True)

In [10]:
unwanted_values = ['foreclosed', 
                   'activated', 
                   'a active',  
                   'pre-foreclosure / auction', 
                   'pre foreclosure auction', 
                   'under contract show', 
                   'under contract   showing', 
                   'active under contract', 
                   '/ auction',  
                   'auction - active', 
                   'active auction', 
                   'auction active', 
                   'under contract backups',
                   'active backup',
                   'pending continue to show',
                   'pending backup wanted',
                   'pending - taking backups'
                   ]

filtered_data = data[data['status'].isin(unwanted_values)]
unwanted_counts = filtered_data['status'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [11]:
print('Количество уникальных значений: ', data['status'].nunique())

Количество уникальных значений:  142


In [12]:
data['status'].value_counts().nlargest(22)

status
for sale                             199571
active                               105650
foreclosure                            7228
new construction                       5475
pending                                4807
pre-foreclosure                        3679
under Contract                         2694
p                                      1488
 / auction                              936
under contract                          692
new                                     690
contingent                              581
auction                                 564
price change                            563
for rent                                398
backup contract                         297
recently sold                           287
contingent finance and inspection       245
active with offer                       243
back on market                          112
option pending                          110
contingent show                         101
Name: count, dtype: int64

In [13]:
print('Доля (%) 22-x наиболее весомых категорий: ' , round((data['status'].value_counts().nlargest(22)/len(data)*100).sum(),1))

Доля (%) 22-x наиболее весомых категорий:  89.2


In [14]:
status_list = list(data['status'].value_counts().nlargest(22).index)
data['status'] = data['status'].progress_apply(lambda x: x if x in status_list else 'other')

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:00<00:00, 749813.88it/s]


In [15]:
print('Количество уникальных значений: ', data['status'].nunique())

Количество уникальных значений:  23


# 2. private pool и PrivatePool 

In [16]:
# выводим количество уникальных значений в столбце private pool
print(f"Количество уникальных значений в столбце '{'private pool'}': {data['private pool'].nunique()}")

# выводим список всех уникальных значений в столбце private pool
unique_values = data['private pool'].unique()
print(f"Все уникальные значения в столбце'{'private pool'}': {unique_values}")

Количество уникальных значений в столбце 'private pool': 1
Все уникальные значения в столбце'private pool': [nan 'Yes']


In [17]:
# выводим количество уникальных значений в столбце PrivatePool
print(f"Количество уникальных значений в столбце '{'PrivatePool'}': {data['PrivatePool'].nunique()}")

# выводим список всех уникальных значений в столбце PrivatePool
unique_values = data['PrivatePool'].unique()
print(f"Все уникальные значения в столбце '{'PrivatePool'}': {unique_values}")

Количество уникальных значений в столбце 'PrivatePool': 2
Все уникальные значения в столбце 'PrivatePool': [nan 'yes' 'Yes']


In [ ]:
# создадим новый признак 'Private_pool' и заполним его нулями
data ['Private_pool'] = 0
# изменим значения столбца 'Private_pool' в соответствии с наличием бассейна
for i in range (0, len(data)):
    if data['private pool'].iloc[i] == 'Yes' or data['PrivatePool'].iloc[i] in ['yes', 'Yes']:
        data ['Private_pool'].iloc[i] = 1

In [19]:
#удалим исходные столбцы
data = data.drop (['private pool','PrivatePool'], axis = 1)

In [20]:
# выводим количество уникальных значений в новом столбце Private_pool
print(f"Количество уникальных значений в столбце '{'Private_pool'}': {data['Private_pool'].nunique()}")

# выводим список всех уникальных значений в столбце Private_pool
unique_values = data['Private_pool'].unique()
print(f"Все уникальные значения в столбце '{'Private_pool'}': {unique_values}")

Количество уникальных значений в столбце 'Private_pool': 2
Все уникальные значения в столбце 'Private_pool': [0 1]


In [21]:
data.head(2)

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,Private_pool
0,active,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,611019,"$418,000",0
1,for sale,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,201916904,"$310,000",0


# 3. propertyType

Признак содержит 1280 уникальных значений (категорий).

In [22]:
print('Доля (%) 50-ти наиболее весомых категорий: ' , round((data['propertyType'].value_counts().nlargest(50)/len(data)*100).sum(),1))

Доля (%) 50-ти наиболее весомых категорий:  88.3


In [23]:
data['propertyType'].value_counts().nlargest(50)

propertyType
single-family home                  92206
Single Family                       62869
Single Family Home                  31728
condo                               25968
lot/land                            20552
Condo                               16561
townhouse                           11464
Land                                10934
multi-family                         7900
Condo/Townhome/Row Home/Co-Op        7701
Townhouse                            6936
Traditional                          5913
coop                                 3266
Multi Family                         2794
High Rise                            1823
Ranch                                1781
mobile/manufactured                  1618
Detached, One Story                  1614
Single Detached, Traditional         1581
Contemporary                         1557
Multi-Family Home                    1501
1 Story                              1234
Colonial                             1205
Mobile / Manufactured

Исправим некоторые дублирования

In [24]:
# Создаем словарь с заменами
replacements_1 = {
    'mobile': 'manufactured home (house)',
    'prefab': 'manufactured home (house)',
    'modular': 'manufactured home (house)',
    'mo2 le': 'manufactured home (house)',
    'mo2le': 'manufactured home (house)',
    'mobile/manufactured': 'manufactured home (house)',
    'mobile / manufactured': 'manufactured home (house)',
    'mfd/mobile home': 'manufactured home (house)',
    'condo/townhome': 'manufactured home (house)',
    'contemporary/modern': 'manufactured home (house)',
    'condo': 'manufactured home (house)',
    'ca2 n': 'cabin',
    'ca2n': 'cabin',
    '2 stories': 'two story',
    '2 story': 'two story',
    '2 stories, traditional': 'two story',
    'farms/ranches': 'ranch',
    'ranch, traditional': 'ranch',
    'mid century': 'midcentury',
    'mid-century': 'midcentury',
    'lot/land': 'land',
    'multi-family': 'multi family',
    'multi-family home': 'multi family',
    'multiple occupancy': 'multi family',
    'single family home': 'single family', 
    'detached, one story': 'single family',
    'single detached': 'single family',
    'two story': 'single family',
    'single detached, contemporary/modern': 'single family',
    'single detached, traditional': 'single family',
    'singlefamilyresidence': 'single family',
    'single-family home': 'single family',
    '1 story': 'one story ',
    '1 story, traditional': 'one story ',
    'traditional, transitional': 'traditional',
}

# Приводим значения столбца "propertyType" к нижнему регистру
data['propertyType'] = data['propertyType'].str.lower()

# Заменяем значения в столбце "propertyType" с помощью словаря замен
data['propertyType'].replace(replacements_1, inplace=True)

Проверяем отсутствие убранных дублирований

In [25]:
unwanted_values = ['mobile', 
                   'prefab', 
                   'modular', 
                   'mo2 le', 
                   'mo2le', 
                   'mobile/manufactured', 
                   'mobile / manufactured', 
                   'mfd/mobile home', 
                   'condo/townhome', 
                   'contemporary/modern', 
                   'condo', 
                   'ca2 n', 
                   'ca2n', 
                   '2 stories', 
                   '2 story', 
                   '2 stories, traditional', 
                   'farms/ranches', 
                   'mid century', 
                   'mid-century', 
                   'lot/land', 
                   'multi-family', 
                   'multi-family home',
                   'multiple occupancy', 
                   'single family home', 
                   'detached, one story', 
                   'single detached', 
                   'single detached, contemporary/modern', 
                   'single detached, traditional', 
                   'singlefamilyresidence',
                   'ranch, traditional',
                   '1 story',
                   '1 story, traditional',
                   'traditional, transitional',
                   'single-family home']

filtered_data = data[data['propertyType'].isin(unwanted_values)]
unwanted_counts = filtered_data['propertyType'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [26]:
print('Количество уникальных значений: ', data['propertyType'].nunique())

Количество уникальных значений:  1249


In [27]:
data['propertyType'].value_counts().nlargest(50)

propertyType
single family                               191457
manufactured home (house)                    47188
land                                         31486
townhouse                                    18400
multi family                                 12374
condo/townhome/row home/co-op                 7701
traditional                                   6039
coop                                          3266
ranch                                         2285
high rise                                     1823
contemporary                                  1557
two story                                     1473
one story                                     1472
colonial                                      1205
apartment                                      922
detached, two story                            638
one story                                      592
transitional                                   560
cooperative                                    535
florida           

In [28]:
print('Доля (%) 43-x наиболее весомых категорий: ' , round((data['propertyType'].value_counts().nlargest(43)/len(data)*100).sum(),1))

Доля (%) 43-x наиболее весомых категорий:  89.0


После 43-й категории вклад остальных категорий незначителен. Заменим их на 'other'


In [29]:
propertyType_list = list(data['propertyType'].value_counts().nlargest(43).index)
data['propertyType'] = data['propertyType'].progress_apply(lambda x: x if x in propertyType_list else 'other')

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:00<00:00, 643615.41it/s]


In [30]:
print('Количество уникальных значений: ', data['propertyType'].nunique())

Количество уникальных значений:  43


# 4. baths

Данный признак имеет тип object, хотя должен быть числовым. Посмотрим первые 15 уникальных значений.

In [31]:
data['baths'].unique()[:15]

array(['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3',
       'Bathrooms: 2', '1,750', '4 Baths', '2 ba', 'Bathrooms: 5',
       '1,000', '7 Baths', '2.0'], dtype=object)

Преобразуем строковое представление числа в числовое значение. Если переданная строка не содержит числовых значений, возвратим NaN.

In [32]:
def get_int(x):
    x = str(x)
    if '/' in x:
        s = x.split('/')
        if '0' in s[0]:
            b = 0
        else:
            b = len(s)
    else: 
        a = re.findall(r"[-+]?(?:\d*\.*\d+)", x.replace(',', '.'))
        if re.findall("\d+", x) != []:
            b = float(a[0])
            return b
        else:
            return np.nan

Добавим новый столбец 'baths' в объект данных 'data' со значениями, полученными из применения функции 'get_int' к столбцу 'baths'.

In [33]:
data['baths'] = data['baths'].progress_apply(get_int)

100%|██████████| 377185/377185 [00:01<00:00, 190965.53it/s]


# 5. homeFacts

Посмотрим, что из себя представляет данный признак

In [34]:
data['homeFacts'][0]

"{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"

Разобьем строку на отдельные словари

In [35]:
facts = ast.literal_eval(data['homeFacts'][0])['atAGlanceFacts']
for fact in facts:
    print (fact)
print (facts)

{'factValue': '2019', 'factLabel': 'Year built'}
{'factValue': '', 'factLabel': 'Remodeled year'}
{'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}
{'factValue': '', 'factLabel': 'Cooling'}
{'factValue': '', 'factLabel': 'Parking'}
{'factValue': None, 'factLabel': 'lotsize'}
{'factValue': '$144', 'factLabel': 'Price/sqft'}
[{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]


Создадим 7 новых признаков

In [36]:
feature_list = ['Cooling','lotsize','Parking','Heating', 'Year built','Remodeled year','Price/sqft']

In [37]:
#Создадим новые столбцы с именами, указанным в списке "feature_list", и заполним все ячейки этого столбца значением np.nan.
for f in feature_list:
    data[f] = np.nan

In [ ]:
#Заполним значениями соответствующие столбцы
for i in range (0, len(data)):
    facts = ast.literal_eval(data['homeFacts'][i])['atAGlanceFacts']
    for fact in facts:
        data[fact['factLabel']].iloc[i] = fact['factValue']

Преобразуем новые признаки

# 5.1 Cooling

In [39]:
print('Доля (%) 15-ти наиболее весомых категорий: ' , round((data['Cooling'].value_counts().nlargest(15)/len(data)*100).sum(),1))

Доля (%) 15-ти наиболее весомых категорий:  92.1


In [40]:
data['Cooling'].value_counts().nlargest(15)

Cooling
Central                                            158754
                                                   120434
Central Air                                         14384
No Data                                             10616
Has Cooling                                          9730
None                                                 7390
Central Electric                                     6154
Wall                                                 4018
Central Gas                                          3573
Central Heating                                      2807
Cooling System                                       2700
Central A/C                                          2051
Other                                                1840
Central A/C (Electric), Central Heat (Gas)           1646
Central A/C (Electric), Central Heat (Electric)      1429
Name: count, dtype: int64

Найдем синонимы

In [41]:


# создаем список всех уникальных значений в столбце
unique_values = data['Cooling'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for '': ['', '1', '2', '3', '5', '4']
Synonyms for 'central air': ['Central Air', 'Central Air,']
Synonyms for 'central a/c': ['Central A/C', 'Central AC']
Synonyms for 'central electric': ['Central Electric', 'Central, Electric', 'Central-Electric']
Synonyms for 'none': ['None', None]
Synonyms for 'ceiling fan, central electric': ['Ceiling Fan, Central Electric', 'Ceiling Fan, Central, Electric', 'Ceiling Fans, Central Electric']
Synonyms for 'yes': ['Yes', 'YES']
Synonyms for 'cooling system': ['Cooling System', 'Cooling System:']
Synonyms for 'None': ['None', None]
Synonyms for 'gas heating': ['Gas Heating', 'Has Heating']
Synonyms for 'window unit(s)': ['Window Unit(s)', 'Window Unit(S)']
Synonyms for 'central, electric': ['Central Electric', 'Central, Electric']
Synonyms for 'ceiling fan, central electric, heat pump': ['Ceiling Fan, Central Electric, Heat Pump', 'Ceiling Fans, Central Electric, Heat Pump']
Synonyms for 'has heating': ['Gas Heating', 'Has Heating']
Synonym

In [42]:
# Создаем словарь с заменами
replacements_Сooling = {
    'central air,': 'central air',
    'cooling system:': 'cooling system',
    'has cooling': 'cooling system',
    'central, electric': 'central electric',
    'central-electric': 'central electric',
    'central ac': 'central a/c',
    '': None,
    'no data': None,
    'none': None
    }

# Приводим значения столбца "status" к нижнему регистру
data['Cooling'] = data['Cooling'].str.lower()

# Заменяем значения в столбце "status" с помощью словаря замен
data['Cooling'].replace(replacements_Сooling, inplace=True)

In [43]:
unwanted_values_Cooling = ['central air,', 
                   'cooling system:', 
                   'has cooling',  
                   'central, electric', 
                   'central-electric', 
                   'central ac', 
                   'no data', 
                   'none', 
                   '',  
                   ]

filtered_data = data[data['Cooling'].isin(unwanted_values_Cooling)]
unwanted_counts = filtered_data['Cooling'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [44]:
Cooling_list = list(data['Cooling'].value_counts().nlargest(15).index)
data['Cooling'] = data['Cooling'].progress_apply(lambda x: x if x in Cooling_list else 'other')

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:00<00:00, 644713.08it/s]


In [45]:
print('Количество уникальных значений: ', data['Cooling'].nunique())

Количество уникальных значений:  15


# 5.2 lotsize

In [46]:
data['lotsize'].unique()[:15]

array([nan, '5828 sqft', '8,626 sqft', '8,220 sqft', '10,019 sqft',
       '680 sqft', '4,996 Sq. Ft.', '8,750 Sq. Ft.', '124582',
       '2,056 sqft', '1.73 acres', '5,715 Sq. Ft.', '10,270 sqft',
       '7,000 sqft lot', '2,130 sqft'], dtype=object)

Используем функцию для извлечения числа

In [47]:
def square(x):
    if type(x) == str:
        a = re.findall(r"[-+]?(?:\d*\.*\d+)", x.replace(',', ''))
        if re.findall("\d+", x) != []:
            b = float(a[0])
            if 'cres' in x:
                b = b*43560.04
            return b
        else:
            return np.nan
    else:
        return np.nan

In [48]:
data['lotsize'] = data['lotsize'].progress_apply(square) 

100%|██████████| 377185/377185 [00:02<00:00, 168822.32it/s]


# 5.3 Parking

In [49]:
data['Parking'].unique()[:15]

array(['', 'Attached Garage', 'Detached Garage',
       'Carport, Attached Garage', '2 spaces', '4 spaces', 'Off Street',
       'None', '0', '1 space', 'No Data', 'Carport', '2', '3 spaces',
       'Double Wide Drive, Oversized Drive'], dtype=object)

In [50]:
print('Количество уникальных значений: ', data['Parking'].nunique())

Количество уникальных значений:  3346


In [51]:
print('Доля (%) 15-ти наиболее весомых категорий: ' , round((data['Parking'].value_counts().nlargest(15)/len(data)*100).sum(),1))

Доля (%) 15-ти наиболее весомых категорий:  91.0


In [52]:
data['Parking'].value_counts().nlargest(15)

Parking
                            171887
Attached Garage              70752
2 spaces                     28063
1 space                      14252
No Data                      13334
Detached Garage              13201
Carport                       7743
3 spaces                      4724
Off Street                    3622
Carport, Attached Garage      3025
1                             2936
4 spaces                      2917
2                             2756
None                          2366
Off street                    1657
Name: count, dtype: int64

In [53]:


# создаем список всех уникальных значений в столбце
unique_values = data['Parking'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for '': ['', '0', '2', '1', '7', '4', '3', '6', '5', '8', '9']
Synonyms for 'attached garage': ['Attached Garage', 'Attached, Garage']
Synonyms for '2 spaces': ['2 spaces', '4 spaces', '3 spaces', '7 spaces', '6 spaces', '8 spaces', '5 spaces', '9 spaces', '12 spaces', '20 spaces', '2+ Spaces', '28 spaces', '32 spaces', '22 spaces', '82 spaces', '24 spaces', '25 spaces', '27 spaces', '29 spaces', '42 spaces', '21 spaces', '23 spaces']
Synonyms for '4 spaces': ['2 spaces', '4 spaces', '3 spaces', '7 spaces', '6 spaces', '8 spaces', '5 spaces', '9 spaces', '14 spaces', '40 spaces', '24 spaces', '44 spaces', '42 spaces', '41 spaces', '43 spaces', '47 spaces']
Synonyms for 'off street': ['Off Street', 'Off street']
Synonyms for 'none': ['None', None, 'none']
Synonyms for '0': ['', '0', '2', '1', '10', '7', '4', '3', '6', '5', '8', '9', '20', '60', '30', '40', '50']
Synonyms for 'carport': ['Carport', '-Carport']
Synonyms for '2': ['', '0', '2', '1', '7', '4', '3', '6', '5', '8', '

In [54]:
# Создаем словарь с заменами
replacements_Parking = {
    'central air,': 'central air',
    'cooling system:': 'cooling system',
    'has cooling': 'cooling system',
    'central, electric': 'central electric',
    'central-electric': 'central electric',
    'central ac': 'central a/c',
    '': None,
    'no data': None,
    'none': None
    }

# Приводим значения столбца "Parking" к нижнему регистру
data['Parking'] = data['Parking'].str.lower()

# Заменяем значения в столбце "Parking" с помощью словаря замен
data['Parking'].replace(replacements_Parking, inplace=True)

In [55]:
unwanted_values_Parking = ['central air,', 
                   'cooling system:', 
                   'has cooling',  
                   'central, electric', 
                   'central-electric', 
                   'central ac', 
                   'no data', 
                   'none', 
                   '',  
                   ]

filtered_data = data[data['Parking'].isin(unwanted_values_Parking)]
unwanted_counts = filtered_data['Parking'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [56]:
print('Количество уникальных значений: ', data['Parking'].nunique())

Количество уникальных значений:  3317


In [57]:
data['Parking'].value_counts().nlargest(15)

Parking
attached garage                     70752
2 spaces                            28063
1 space                             14252
detached garage                     13201
carport                              7743
off street                           5279
3 spaces                             4724
carport, attached garage             3025
1                                    2936
4 spaces                             2917
2                                    2756
on street                            1707
attached garage, detached garage     1354
0                                    1114
attached garage, carport              993
Name: count, dtype: int64

In [58]:
Parking_list = list(data['Parking'].value_counts().nlargest(15).index)
data['Parking'] = data['Parking'].progress_apply(lambda x: x if x in Parking_list else 'other')

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:00<00:00, 625467.93it/s]


In [59]:
print('Количество уникальных значений: ', data['Parking'].nunique())

Количество уникальных значений:  16


# 5.4 Heating

In [60]:
data['Heating'].unique()[:15]

array(['Central A/C, Heat Pump', '', 'Forced Air', 'Electric, Heat Pump',
       'Gas', 'Central Electric', 'Forced air', 'Other',
       'Forced air, Heat pump', 'Central Air', 'Electric', 'Heat Pump',
       'Radiant', 'Baseboard', 'Central Furnace'], dtype=object)

In [61]:
print('Количество уникальных значений: ', data['Heating'].nunique())

Количество уникальных значений:  1984


In [62]:
print('Доля (%) 15-ти наиболее весомых категорий: ' , round((data['Heating'].value_counts().nlargest(15)/len(data)*100).sum(),1))

Доля (%) 15-ти наиболее весомых категорий:  90.0


In [63]:
data['Heating'].value_counts().nlargest(15)

Heating
                     105799
Forced Air            82807
Forced air            51506
Other                 29623
Electric              10211
Gas                    9296
No Data                8611
Central Air            7814
Central Electric       7112
Central                6247
Heat Pump              6104
Central, Electric      4253
Baseboard              3815
Wall                   3301
Electric Heat          3064
Name: count, dtype: int64

In [64]:


# создаем список всех уникальных значений в столбце
unique_values = data['Heating'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for '': ['', '3', '1', '5', '2']
Synonyms for 'forced air': ['Forced Air', 'Forced air', 'ForcedAir', 'Forced-Air']
Synonyms for 'electric, heat pump': ['Electric, Heat Pump', 'ELECTRIC HEAT PUMP', 'Electric Heat Pump']
Synonyms for 'gas': ['Gas', 'gas', 'GAS']
Synonyms for 'central electric': ['Central Electric', 'Central, Electric', 'Central-Electric']
Synonyms for 'forced air, heat pump': ['Forced air, Heat pump', 'Forced Air, Heat Pump']
Synonyms for 'electric': ['Electric', 'electric']
Synonyms for 'heat pump': ['Heat Pump', 'Heat pump', 'HeatPump']
Synonyms for 'central, electric': ['Central Electric', 'Central, Electric']
Synonyms for 'lower level-gas': ['Lower level-Gas', 'Lower Level-Gas', 'lower level-Gas']
Synonyms for 'heating system': ['Heating System', 'Heating System:']
Synonyms for 'None': [None, 'None']
Synonyms for 'central, gas': ['Central, Gas', 'Central Gas']
Synonyms for 'central electric, zoned': ['Central Electric, Zoned', 'Central, Electric, Zoned']
Sy

In [65]:
# Создаем словарь с заменами
replacements_Heating = {
    'forcedair,': 'forced air',
    'forced-air': 'forced air',
    'central, electric': 'central electric',
    'central-electric': 'central electric',
    'heatpump': 'heat pump',
    '': None,
    'no data': None,
    'none': None
    }

# Приводим значения столбца "status" к нижнему регистру
data['Heating'] = data['Heating'].str.lower()

# Заменяем значения в столбце "status" с помощью словаря замен
data['Heating'].replace(replacements_Heating, inplace=True)

In [66]:
unwanted_values_Heating = ['forcedair,', 
                   'forced-air', 
                   'central, electric',   
                   'central-electric', 
                   'heatpump', 
                   'no data', 
                   'none', 
                   '',  
                   ]

filtered_data = data[data['Heating'].isin(unwanted_values_Heating)]
unwanted_counts = filtered_data['Heating'].value_counts()

print(unwanted_counts)

Series([], Name: count, dtype: int64)


In [67]:
print('Количество уникальных значений: ', data['Heating'].nunique())

Количество уникальных значений:  1911


In [68]:
data['Heating'].value_counts().nlargest(15)

Heating
forced air                     134314
other                           29623
central electric                11368
electric                        10217
gas                              9299
heat pump                        8855
central air                      7814
central                          6247
baseboard                        3815
wall                             3301
electric heat                    3064
heating system                   2709
forced air, heat pump            1767
radiant                          1485
central air, ceiling fan(s)      1432
Name: count, dtype: int64

In [69]:
Heating_list = list(data['Heating'].value_counts().nlargest(15).index)
data['Heating'] = data['Heating'].progress_apply(lambda x: x if x in Heating_list else 'other')

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:00<00:00, 622373.96it/s]


In [70]:
print('Количество уникальных значений: ', data['Heating'].nunique())

Количество уникальных значений:  15


# 5.5 Year built

In [71]:
data['Year built'].unique()[:15]

array(['2019', '1961', '2006', '', '1920', '1976', '1970', '1965', '2015',
       '1996', '1982', '1905', '2008', '1899', '2016'], dtype=object)

Cоздадим функцию для извлечения чисел, включая float

In [72]:
def get_float(string):
    a = re.findall(r"[-+]?(?:\d*\.*\d+)", str(string).replace(',', '.'))
    if a != []:
        b = float(a[0])
        return b
    else:
        return np.nan

In [73]:
data['Year built'] = data['Year built'].progress_apply(get_float)

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:01<00:00, 261914.29it/s]


Изменим год постройки на возраст дома

In [74]:
data['Year built'] = data['Year built'].progress_apply(lambda x: 2022 - x)

100%|██████████| 377185/377185 [00:00<00:00, 749819.57it/s]


# 5.6 Remodeled year

Повторим

In [75]:
data['Remodeled year'] = data['Remodeled year'].progress_apply(get_float)
data['Remodeled year'] = data['Remodeled year'].progress_apply(lambda x: 2022 - x)

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:00<00:00, 706285.76it/s]


# 5.7 Price/sqft

In [76]:
data['Price/sqft'] = data['Price/sqft'].progress_apply(get_float)

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:01<00:00, 220737.72it/s]


In [77]:
data = data.drop('homeFacts', axis = 1)

# 6 city

In [78]:
data['city'].unique()[:15]

array(['Southern Pines', 'Spokane Valley', 'Los Angeles', 'Dallas',
       'Palm Bay', 'Philadelphia', 'POINCIANA', 'Memphis', 'Mason City',
       'Houston', 'Flushing', 'Pembroke Pines', 'Eugene', 'Brooklyn',
       'Fort Lauderdale'], dtype=object)

In [79]:
print('Количество уникальных значений: ', data['city'].nunique())

Количество уникальных значений:  2026


In [80]:


# создаем список всех уникальных значений в столбце
unique_values = data['city'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for 'los angeles': ['Los Angeles', 'Los angeles']
Synonyms for 'dallas': ['Dallas', 'DALLAS']
Synonyms for 'poinciana': ['POINCIANA', 'Poinciana']
Synonyms for 'new york': ['New York', 'New york']
Synonyms for 'bend': ['Bend', 'BEND']
Synonyms for 'spring': ['Spring', 'Springs']
Synonyms for 'miami': ['Miami', 'MIAMI']
Synonyms for 'davie': ['Davie', 'Davis']
Synonyms for 'tampa': ['Tampa', 'TAMPA']
Synonyms for 'jacksonville': ['Jacksonville', 'JACKSONVILLE']
Synonyms for 'charlotte': ['Charlotte', 'CHARLOTTE']
Synonyms for 'rotonda west': ['ROTONDA WEST', 'Rotonda West']
Synonyms for 'weston': ['Weston', 'Reston']
Synonyms for 'fort worth': ['Fort Worth', 'Fort worth']
Synonyms for 'temple terrace': ['Temple Terrace', 'TEMPLE TERRACE']
Synonyms for 'atlanta': ['Atlanta', 'Atlaanta']
Synonyms for 'lakewood': ['Lakewood', 'LAKEWOOD']
Synonyms for 'seattle': ['Seattle', 'SEATTLE']
Synonyms for 'port charlotte': ['PORT CHARLOTTE', 'Port Charlotte']
Synonyms for 'saint johns': ['

Необходимо привести названия городов к одинаковому регистру

In [81]:
data['city'] = data['city'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [82]:


# создаем список всех уникальных значений в столбце
unique_values = data['city'].unique()

# создаем список всех уникальных значений в столбце, приведенных к нижнему регистру
lowercase_values = [str(value).lower() if isinstance(value, str) else str(value) for value in unique_values]

# создаем словарь синонимов (ключ - это нормализованное значение, значение - это список всех значений, которые считаются синонимами)
synonyms_dict = {}

for value in unique_values:
    # нормализуем текущее значение
    normalized_value = str(value).lower() if isinstance(value, str) else str(value)

    # ищем все значения, которые считаются синонимами для текущего значения
    synonyms = [v for v in unique_values if distance(str(v).lower() if isinstance(v, str) else str(v), normalized_value) <= 1]

    # добавляем текущее значение и его синонимы в словарь
    synonyms_dict[normalized_value] = synonyms

# выводим все найденные синонимы
for key, value in synonyms_dict.items():
    if len(value) > 1:
        print(f"Synonyms for '{key}': {value}")

Synonyms for 'spring': ['spring', 'springs']
Synonyms for 'davie': ['davie', 'davis']
Synonyms for 'weston': ['weston', 'reston']
Synonyms for 'atlanta': ['atlanta', 'atlaanta']
Synonyms for 'valley view': ['valley view', 'valleyview']
Synonyms for 'mckinney': ['mckinney', 'mc kinney']
Synonyms for 'austin': ['austin', 'tustin']
Synonyms for 'nashville': ['nashville', 'ashville']
Synonyms for 'st petersburg': ['st petersburg', 'st. petersburg']
Synonyms for 'bryan': ['bryan', 'bryant']
Synonyms for 'irving': ['irving', 'irvine']
Synonyms for 'swanton': ['swanton', 'stanton']
Synonyms for 'asheville': ['asheville', 'ashville']
Synonyms for 'brooksville': ['brooksville', 'crooksville']
Synonyms for 'forest hills': ['forest hills', 'forest hill']
Synonyms for 'ashville': ['nashville', 'asheville', 'ashville']
Synonyms for 'irvine': ['irving', 'irvine']
Synonyms for 'chino': ['chino', 'chico', 'china']
Synonyms for 'soddy daisy': ['soddy daisy', 'soddy-daisy']
Synonyms for 'bellingham': ['

In [83]:
# Создаем словарь с заменами
replacements_city = {
    'springs,': 'spring',
    'davis': 'davie',
    'reston': 'weston',
    'atlaanta': 'atlanta',
    'valleyview': 'valley view',
    'mc kinney': 'mckinney',
    'tustin': 'austin',
    'ashville': 'nashville',
    'st. petersburg': 'st petersburg',
    'bryant': 'bryan',
    'irvine': 'irving',
    'stanton': 'swanton',
    'asheville': 'nashville',
    'chico': 'chino',
    'china': 'chino',
    'soddy-daisy': 'soddy daisy',
    'belllingham': 'bellingham',
    'st. johns': 'st johns',
    'aventura': 'ventura',
    'oklawaha': 'ocklawaha',
    'clifton': 'clinton',
    'silver spring': 'silver springs',
    'rossville': 'roseville',
    'arrington': 'arlington',
    'commercecity': 'commerce city',
    'denton': 'renton',
    'trenton': 'renton',
    'reston': 'renton',
    'inglewood': 'englewood',
    'panton': 'canton',
    'canyon': 'canton',
    'delton': 'renton',
    'youngtown': 'youngstown',
    'morrison': 'morristown',
    'morriston': 'morristown',
    'reading': 'redding',
    'fairfield twp.': 'fairfield twp',
    'silver springs': 'silver spring',
    'opa-locka': 'opa locka',
    'wheatridge': 'wheat ridge',
    'alden': 'allen',
    'maxville': 'mayville',
    'silver lakes': 'silver lake',
    'willsboro': 'hillsboro',
    'bedford': 'redford',
    'medford': 'redford',
    'milwaukee': 'milwaukie',
    'federal heights': 'federalheights',
    'bee caves': 'bee cave',
    'dalton': 'dayton',
    'stow': 'stowe',
    'forest hills': 'forest hill',
    'brookpark': 'brook park',
    'powell': 'lowell',
    'mercer': 'merced',
    'roseville': 'rossville',
    'doctor philips': 'doctor phillips',
    'arrington': 'barrington',
    'mc allen': 'mcallen',
    'st albans city': 'st. albans city',
    'deltona': 'renton',
    'belton': 'renton',
    'dolton': 'renton',
    'ft. worth': 'ft worth',
    'mill creek': 'millcreek',
    'st. george': 'st george',
    'onon hill': 'oxon hill',
    'arden': 'allen',
    'belmond': 'belmont',
    'mcgregor': 'mc gregor',
    'lake forest': 'wake forest',
    'nome': 'rome',
    'rowe': 'rome',
    'ruston': 'reston',
    'leona valley': 'leon valley',
    'troy': 'roy',
    'biola': 'iola',
    'highlands': 'highland',
    'green acres': 'greenacres',
    'downs': 'dows',
    'bryant': 'brant',
    'unincorporated broward county': 'un-incorporated broward county',
    '': None,
    'no data': None,
    'none': None
    }


# Заменяем значения в столбце "sity" с помощью словаря замен
data['city'].replace(replacements_city, inplace=True)

In [84]:
print('Количество уникальных значений: ', data['city'].nunique())

Количество уникальных значений:  1836


In [85]:
print('Доля (%) 300 наиболее весомых категорий: ' , round((data['city'].value_counts().nlargest(300)/len(data)*100).sum(),1))

Доля (%) 300 наиболее весомых категорий:  90.5


In [86]:
city_list = list(data['city'].value_counts().nlargest(300).index)
data['city'] = data['city'].progress_apply(lambda x: x if x in city_list else 'other')

100%|██████████| 377185/377185 [00:01<00:00, 351824.75it/s]


# 7 schools

Посмотрим на данные

In [87]:
data['schools'][2]

"[{'rating': ['8/10', '4/10', '8/10'], 'data': {'Distance': ['1.19mi', '2.06mi', '2.63mi'], 'Grades': ['6-8', 'K-5', '9-12']}, 'name': ['Paul Revere Middle School', 'Brentwood Science School', 'Palisades Charter High School']}]"

Разобьем строку на отдельные словари

In [88]:
facts = ast.literal_eval(data['schools'][0])
for fact in facts:
    print (fact)
print (facts)

{'rating': ['4', '4', '7', 'NR', '4', '7', 'NR', 'NR'], 'data': {'Distance': ['2.7 mi', '3.6 mi', '5.1 mi', '4.0 mi', '10.5 mi', '12.6 mi', '2.7 mi', '3.1 mi'], 'Grades': ['3–5', '6–8', '9–12', 'PK–2', '6–8', '9–12', 'PK–5', 'K–12']}, 'name': ['Southern Pines Elementary School', 'Southern Middle School', 'Pinecrest High School', 'Southern Pines Primary School', "Crain's Creek Middle School", 'Union Pines High School', 'Episcopal Day Private School', 'Calvary Christian Private School']}
[{'rating': ['4', '4', '7', 'NR', '4', '7', 'NR', 'NR'], 'data': {'Distance': ['2.7 mi', '3.6 mi', '5.1 mi', '4.0 mi', '10.5 mi', '12.6 mi', '2.7 mi', '3.1 mi'], 'Grades': ['3–5', '6–8', '9–12', 'PK–2', '6–8', '9–12', 'PK–5', 'K–12']}, 'name': ['Southern Pines Elementary School', 'Southern Middle School', 'Pinecrest High School', 'Southern Pines Primary School', "Crain's Creek Middle School", 'Union Pines High School', 'Episcopal Day Private School', 'Calvary Christian Private School']}]


Cоздадим функцию для извлечения характеристик, принимая во внимание формат: количество школ, а также минимальный, средний и максимальный рейтинг школ и расстояние до школы

In [89]:
def schools_feature (x):
    
    x =  ast.literal_eval(x)
    schools = pd.json_normalize(x)
            
    len_list = []
    for i in range(0,4):
        len_list.append(len(schools.iloc[0][i]))
    mx = max(len_list)
    
    for col in list(schools.columns):
        while len(schools[col][0]) < mx:
            schools[col].iloc[0].append(np.nan)
            
    schools = schools.explode(['rating','name','data.Distance', 'data.Grades'])
       

    if len(schools['rating']) == 1 and schools['rating'][0] is np.nan:
        Min_rating, Max_rating, Mean_rating = np.nan, np.nan, np.nan
    else: 
        schools['rating'] = schools['rating'].apply(get_float)
        Min_rating, Max_rating, Mean_rating = schools['rating'].min(), schools['rating'].max(), schools['rating'].mean()

        
    if len(schools['data.Distance']) == 1 and schools['data.Distance'][0] is np.nan:
        Min_distance, Max_distance, Mean_distance = np.nan, np.nan, np.nan
    else: 
        schools['data.Distance'] = schools['data.Distance'].apply(get_float)
        Min_distance, Max_distance, Mean_distance = schools['data.Distance'].min(), schools['data.Distance'].max(), schools['data.Distance'].mean()

    return len(schools), Min_rating, Max_rating, Mean_rating, Min_distance, Max_distance, Mean_distance

    
school_feature_list = ['Number of schools', 'Min rating', 'Max rating', 'Mean rating', 'Min distance', 'Max distance', 'Mean distance']
  
data [['Number of schools', 'Min rating','Max rating','Mean rating','Min distance','Max distance','Mean distance']] = data['schools'].progress_apply(schools_feature).apply(pd.Series)

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [54:01<00:00, 116.37it/s] 


In [90]:
data = data.drop('schools', axis =1)

In [92]:
data.to_csv('data_data.csv')

# 8 sqft

In [93]:
data['sqft'].unique()[:15]

array(['2900', '1,947 sqft', '3,000 sqft', '6,457 sqft', nan, '897 sqft',
       '1,507', '3588', '1,930', '1,300 sqft', '3,130', '2,839 sqft',
       'Total interior livable area: 1,820 sqft', '2,454', '2,203'],
      dtype=object)

In [94]:
data['sqft'] = data['sqft'].progress_apply(get_float)

  0%|          | 0/377185 [00:00<?, ?it/s]

100%|██████████| 377185/377185 [00:01<00:00, 227613.27it/s]


In [95]:
data['sqft'].unique()[:15]

array([2.900e+03, 1.947e+00, 3.000e+00, 6.457e+00,       nan, 8.970e+02,
       1.507e+00, 3.588e+03, 1.930e+00, 1.300e+00, 3.130e+00, 2.839e+00,
       1.820e+00, 2.454e+00, 2.203e+00])

# 9 zipcode

In [96]:
data['zipcode'].unique()[:15]

array(['28387', '99216', '90049', '75205', '32908', '19145', '34759',
       '38115', '50401', '77080', '11354', '77068', '33028', '97401',
       '77084'], dtype=object)

In [97]:
print('Количество уникальных значений: ', data['zipcode'].nunique())

Количество уникальных значений:  4549


# 10 beds

In [98]:
data['beds'].unique()[:100]

array(['4', '3 Beds', '5 Beds', nan, '2 Beds', '3', '4 Beds', '3 bd',
       'Baths', '4 bd', '5 bd', '8 Beds', '2', '2 bd', '2.0', '5', '1',
       '3.0', '6 Beds', 'Bath', '12 Beds', '6', '14', '0.25 acres',
       '6 bd', '-- bd', '7', '5.0', '4.0', ' ', '7 Beds', '1 bd',
       '9 Beds', '0.44 acres', '0', '1.43 acres', '0.32 acres',
       '8,637 sqft', '7.0', '8', '16 Beds', '9.7 acres', '1.0', '16 bd',
       '10,310 sqft', '6,250 sqft', '3.02 acres', '2 acres', '5,000 sqft',
       '0.96 acres', '6.0', '13 bd', '6.31 acres', '0.6 acres', '9',
       '5,662 sqft', '2,200 sqft', '10', '18 Beds', '10 Beds',
       '0.46 acres', '0.59 acres', '20 Beds', '0.0', '2,874 sqft',
       '0.26 acres', '10 bd', '32', '0.4 acres', '2,178 sqft',
       '6,351 sqft', '4,356 sqft', '5 acres', '0.55 acres', '5,501 sqft',
       '8,001 sqft', '0.7 acres', '1.69 acres', '0.88 acres',
       '0.34 acres', '13 Beds', '1.5 acres', '0.97 acres', '7,405 sqft',
       '7 bd', '8.43 acres', '1,502 sqft'

In [99]:
data['beds'] = data['beds'].progress_apply(get_float)
data['beds'] = data['beds'].progress_apply(lambda x: x if  (1<= x <1000 and x.is_integer()) else np.nan)

100%|██████████| 377185/377185 [00:00<00:00, 766587.73it/s]


In [100]:
data['beds'].unique()[:100]

array([  4.,   3.,   5.,  nan,   2.,   8.,   1.,   6.,  12.,  14.,   7.,
         9.,  16.,  13.,  10.,  18.,  20.,  32.,  11.,  26.,  64., 144.,
        22.,  24.,  15.,  28.,  60.,  36.,  19.,  48.,  25.,  17.,  27.,
        37.,  40.,  47.,  23.,  44.,  30.,  33.,  38.,  31.,  34., 831.,
       248.,  99., 184.,  78., 542.,  53.,  50.,  51., 102.,  21., 106.,
        75.,  29., 640., 449., 448., 100.,  39.,  52.,  42., 150., 540.,
        46., 871.,  76.,  35.,  88., 840.])

# 11 state

In [101]:
data['state'].unique()[:15]

array(['NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR', 'DC',
       'NV', 'AZ', 'GA', 'IL'], dtype=object)

In [102]:
print('Количество уникальных значений: ', data['state'].nunique())

Количество уникальных значений:  39


# 12 stories

In [103]:
data['stories'].unique()[:100]

array([nan, '2.0', '1.0', '3.0', 'One', '2', 'Multi/Split', '4.0', '0.0',
       '0', 'One Level', '1', '9.0', '3', '1 Level, Site Built',
       'One Story', '3.00', '1.00', '14.0', 'Two', '3+', '1 Story', '5.0',
       '2 Story', 'Ranch/1 Story', 'Condominium', 'Stories/Levels', '7.0',
       '2 Level, Site Built', '2 Level', '15', '3 Level, Site Built', '4',
       '22.0', '2.00', '6.0', '1.0000', 'Lot', '3 Story', 'Three Or More',
       '1.5', '1 Level', 'Two Story or More', 'Site Built, Tri-Level',
       '54.0', '23', 'Farm House', '8.0', '16.0', '1.50', '18', '9', '21',
       '8', '12.0', 'Split Level w/ Sub', '11.0', '18.0', '1.5 Stories',
       '7', '11', 'Townhouse', '12', '21.0', '16', '1.5 Story/Basement',
       '28.0', 'Traditional', '2.5 Story', '17', '2.0000', '63.0',
       'Acreage', 'Ground Level, One', '6', 'Split Foyer', '2 Stories',
       '27.0', '19.0', '2.50', '1.30', '2 Story/Basement', 'Split Level',
       '1.5 Story', '1.5 Level', '2 Or More Stories',
  

В признаке разношерстные данные, к тому же пропуски составляют почти 40%. Удалим данный признак.

In [104]:
data = data.drop('stories', axis=1)

# 13 mls-id и MLsId

Данный признак не несет информации, влияющей на стоимость жилья, поэтому удалим его.

In [105]:
data = data.drop('mls-id', axis=1)

In [106]:
data = data.drop('MlsId', axis=1)

# 14 street

In [107]:
data = data.drop('street', axis=1)

# 15 fireplace

Данный признак содержит почти 73% пропусков. Удалим его.

In [108]:
data = data.drop('fireplace', axis=1)

# 16 target

In [109]:
data['target'].unique()[:50]

array(['$418,000', '$310,000', '$2,895,000', '$2,395,000', '$5,000',
       '$209,000', '181,500', '68,000', '$244,900', '$311,995',
       '$669,000', '260,000', '$525,000', '$499,900', '$168,800',
       '1,650,000', '335,000', '2,650,000', '$365,000', '$626,000',
       '$375,000', '$3,500,000', '579,000', '$499,007', '$182,000',
       '$3,749,000', '799,000', '$559,000', '$830,000', '$1,195,000',
       '$262,000', '$204,800', '$179,000', '$105,000', '$260,000',
       '$284,900', '$495,000', '$117,900', '$385,000', '54,000',
       '$1,100,000', '850,000', '$620,000', '$125,000', '$499,000',
       '$1,429,000', '$233,990+', '125,000', '$275,000', '$27,000'],
      dtype=object)

In [110]:
print('Количество уникальных значений: ', data['target'].nunique())

Количество уникальных значений:  43939


In [111]:
data['target'] = data['target'].progress_apply(get_float)

100%|██████████| 377185/377185 [00:01<00:00, 307381.12it/s]


In [112]:
data['target'].unique()[:50]

array([418.   , 310.   ,   2.895,   2.395,   5.   , 209.   , 181.5  ,
        68.   , 244.9  , 311.995, 669.   , 260.   , 525.   , 499.9  ,
       168.8  ,   1.65 , 335.   ,   2.65 , 365.   , 626.   , 375.   ,
         3.5  , 579.   , 499.007, 182.   ,   3.749, 799.   , 559.   ,
       830.   ,   1.195, 262.   , 204.8  , 179.   , 105.   , 284.9  ,
       495.   , 117.9  , 385.   ,  54.   ,   1.1  , 850.   , 620.   ,
       125.   , 499.   ,   1.429, 233.99 , 275.   ,  27.   , 598.   ,
         1.78 ])

In [113]:
data.head()

,status,propertyType,baths,city,sqft,zipcode,beds,state,target,Private_pool,...,Year built,Remodeled year,Price/sqft,Number of schools,Min rating,Max rating,Mean rating,Min distance,Max distance,Mean distance
0,active,single family,3.5,southern pines,2900.000,28387,4.0,NC,418.000,0,...,3.0,NaN,144.0,8.0,4.0,7.0,5.200000,2.70,12.60,5.537500
1,for sale,single family,3.0,other,1.947,99216,3.0,WA,310.000,0,...,3.0,NaN,159.0,3.0,4.0,10.0,6.000000,1.01,1.65,1.326667
2,for sale,single family,2.0,los angeles,3.000,90049,3.0,CA,2.895,1,...,61.0,55.0,965.0,3.0,4.0,8.0,6.666667,1.19,2.63,1.960000
3,for sale,single family,8.0,dallas,6.457,75205,5.0,TX,2.395,0,...,16.0,16.0,371.0,4.0,9.0,10.0,9.250000,0.10,1.05,0.752500
4,for sale,land,NaN,palm bay,NaN,32908,NaN,FL,5.000,0,...,NaN,NaN,NaN,3.0,4.0,5.0,4.666667,3.03,5.96,4.080000


In [114]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   status             377185 non-null  object 
 1   propertyType       377185 non-null  object 
 2   baths              269278 non-null  float64
 3   city               377185 non-null  object 
 4   sqft               335788 non-null  float64
 5   zipcode            377185 non-null  object 
 6   beds               264122 non-null  float64
 7   state              377185 non-null  object 
 8   target             374704 non-null  float64
 9   Private_pool       377185 non-null  int64  
 10  Cooling            377185 non-null  object 
 11  lotsize            281330 non-null  float64
 12  Parking            377185 non-null  object 
 13  Heating            377185 non-null  object 
 14  Year built         313589 non-null  float64
 15  Remodeled year     151075 non-null  float64
 16  Pr

In [115]:
data.to_csv('df.csv')